In [1]:
# Define Imports
import os
import googlemaps
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
import pydeck as pdk
from dash import Dash, dcc, html
from dotenv import load_dotenv
import numpy as np

In [2]:
# Load ENV variables
load_dotenv(dotenv_path='../../.env')

True

In [4]:
# Preprocess the Submissions Dataset
df = pd.read_csv("../../data/submissions.csv")
df = df.loc[~df.loc[:, "initiator_address"].isna()]
df

,submission_code,submission_date,submission_status,submission_description,initiator_id,initiator_legalname,initiator_address,initiator_region,item_amount,item_classification
0,010624-040A8668,2024-06-01T17:36:27.658298Z,pending,Харчоблок базової кухні,4394869,ВЕЛИКОБІРКІВСЬКА СЕЛИЩНА РАДА,"Тернопільська обл., Тернопільський р-н, селище...",Тернопільська,3967203.0,kitchen
1,020624-2C34FAB7,2024-06-02T16:32:18.130273Z,pending,харчоблок,2146222,"УПРАВЛІННЯ ОСВІТИ, МОЛОДІ ТА СПОРТУ ЛОЗІВСЬКОЇ...","Харківська обл., місто Лозова, МІКРОРАЙОН 1, б...",Харківська,9607874.0,kitchen
2,020624-9F163A6A,2024-06-02T18:06:32.388695Z,pending,харчоблок,2146222,"УПРАВЛІННЯ ОСВІТИ, МОЛОДІ ТА СПОРТУ ЛОЗІВСЬКОЇ...","Харківська обл., місто Лозова, МІКРОРАЙОН 1, б...",Харківська,16569763.0,kitchen
3,020624-D8610EBB,2024-06-02T17:09:51.508281Z,pending,харчоблок,2146222,"УПРАВЛІННЯ ОСВІТИ, МОЛОДІ ТА СПОРТУ ЛОЗІВСЬКОЇ...","Харківська обл., місто Лозова, МІКРОРАЙОН 1, б...",Харківська,20051019.0,kitchen
4,020624-E1B2F3B0,2024-06-02T16:20:55.533024Z,pending,харчоблок,2146222,"УПРАВЛІННЯ ОСВІТИ, МОЛОДІ ТА СПОРТУ ЛОЗІВСЬКОЇ...","Харківська обл., місто Лозова, МІКРОРАЙОН 1, б...",Харківська,16166182.0,kitchen
...,...,...,...,...,...,...,...,...,...,...
483,310524-F8C525A4,2024-05-31T11:04:36.688246Z,withdrawn,Харчоблок Сварицевицького ліцею,44153430,"ВІДДІЛ ОСВІТИ, КУЛЬТУРИ, ТУРИЗМУ, МОЛОДІ ТА СП...","Рівненська обл., Дубровицький р-н, місто Дубро...",Рівненська,8327800.0,kitchen
484,310524-F9A91731,2024-05-31T15:20:47.521727Z,approval,Капітальний ремонт харчоблоку (їдальні) ліцею ...,44084980,ВІДДІЛ ОСВІТИ ПІДГОРОДНЕНСЬКОЇ МІСЬКОЇ РАДИ ДН...,"Дніпропетровська обл., Дніпровський р-н, місто...",Дніпропетровська,11745067.5,kitchen
485,310524-F9C0426D,2024-05-31T18:11:46.185941Z,pending,Капітальний ремонт харчоблоку Середньоберезівс...,4354255,ЯБЛУНІВСЬКА СЕЛИЩНА РАДА КОСІВСЬКОГО РАЙОНУ ІВ...,"Івано-Франківська обл., Косівський р-н, селище...",Івано-Франківська,870410.0,kitchen
486,310524-FD0808D6,2024-05-31T11:34:02.120912Z,withdrawn,Капітальний ремонт харчоблоку,4412567,ПОНОРНИЦЬКА СЕЛИЩНА РАДА,"Чернігівська обл., Новгород-Сіверський р-н, се...",Чернігівська,3122988.0,kitchen


In [6]:
# Load Geocoded Regions
regions_geocoded_df = pd.read_csv('../../data/regions_geocoded.csv')
# Group by the specified columns and count submissions
groupby_columns = [
    'initiator_region',
    'item_classification',
    'submission_status'
]
grouper_count_column = 'number_of_submissions'
submissions_grouped_df = df.groupby(
        groupby_columns,
        as_index=False
    ).count(
    ).rename(
        columns={'submission_code': grouper_count_column}
    )
submissions_grouped_df = submissions_grouped_df.loc[
    :,
    submissions_grouped_df.columns.isin(
        groupby_columns + [grouper_count_column]
    )]
submissions_grouped_df = submissions_grouped_df.merge(regions_geocoded_df, on='initiator_region', how='left')
submissions_grouped_df = submissions_grouped_df.sort_values(by=["number_of_submissions", "item_classification", "submission_status"], ascending=False)
submissions_grouped_df = submissions_grouped_df.dropna()
submissions_grouped_df

,initiator_region,item_classification,submission_status,number_of_submissions,latitude,longitude
37,Львівська,kitchen,pending,21,49.839683,24.029717
32,Кіровоградська,kitchen,pending,17,48.507933,32.262317
71,Харківська,shelter,approval,16,50.002013,36.307399
28,Київська,kitchen,pending,16,50.052951,30.766713
46,Одеська,kitchen,pending,16,46.484583,30.732600
...,...,...,...,...,...,...
36,Львівська,kitchen,declined,1,49.839683,24.029717
0,Івано-Франківська,kitchen,approval,1,48.920062,24.708916
17,Житомирська,kitchen,approval,1,50.254650,28.658667
27,Київська,kitchen,approval,1,50.052951,30.766713


In [7]:
# Save Preprocessed Dataset
submissions_grouped_df.to_csv("../../data/submissions_silver.csv", index=False)